In [1]:
import os
import pymysql
import pandas as pd
import numpy as np
import re
from pandas.io import sql
import MySQLdb
from sqlalchemy import create_engine

In [2]:
from pandas.io import sql
import MySQLdb

conn = MySQLdb.connect(
    host="localhost",
    port=int(3306),
    user="pythonlink",
    passwd="pythonlinkp",
    db="stock_forecast_v1",
    charset='utf8mb4'
)

data = pd.read_sql_query("SELECT * FROM delta_mergedstatements500_withpricechange",
    conn)
data

Company        Time  Total Assets  Total Liabilities  Total Equity  \
0          A  2011-10-01        -639.0            -1719.0        1080.0   
1          A  2012-10-01        1479.0              610.0         869.0   
2          A  2013-10-01         150.0               46.0         104.0   
3          A  2014-10-01         129.0              114.0          15.0   
4          A  2015-10-01       -3336.0            -2202.0       -1134.0   
5          A  2016-10-01         315.0              239.0          76.0   
6          A  2017-10-01         632.0               43.0         589.0   
7          A  2018-10-01         115.0              379.0        -264.0   
8          A  2019-10-01         911.0              734.0         177.0   
9        AAL  2011-12-01       -1240.0             1926.0       -3166.0   
10       AAL  2012-12-01        -338.0              538.0        -876.0   
11       AAL  2013-12-01       18768.0            13512.0        5256.0   
12       AAL  2014-12-01         947.0            -3805.0        4752.0   
13       AAL  2015-12-01        5190.0             1576.0        3614.0   
14       AAL  2016-12-01        2859.0             4709.0       -1850.0   
15       AAL  2017-12-01        1511.0             6076.0       -4565.0   
16       AAL  2018-12-01        7795.0             7184.0         611.0   
17       AAL  2019-12-01        -585.0             -636.0          51.0   
18       AAP  2011-12-01         301.6              493.0        -191.5   
19       AAP  2012-12-01         958.0              595.3         362.8   
20       AAP  2013-12-01         951.0              645.5         305.5   
21       AAP  2015-01-01        2397.6             1910.8         486.7   
22       AAP  2016-01-01         165.3             -292.3         457.7   
23       AAP  2016-12-01         187.3             -268.3         455.6   
24       AAP  2017-12-01         167.3             -331.7         499.0   
25       AAP  2018-12-01         558.3              422.7         135.6   
26       AAP  2019-12-01        2207.9             2209.6          -1.7   
27      AAPL  2011-09-01       41188.0            12364.0       28824.0   
28      AAPL  2012-09-01       59693.0            18098.0       41595.0   
29      AAPL  2013-09-01       30936.0            25597.0        5339.0   
...      ...         ...           ...                ...           ...   
4470     XRX  2017-12-01       -2105.0            -2651.0         546.0   
4471     XRX  2018-12-01       -1072.0             -818.0        -254.0   
4472     XRX  2019-12-01         173.0             -382.0         555.0   
4473    YORW  2011-12-01          14.3               10.3           4.0   
4474    YORW  2012-12-01           8.3                3.7           4.5   
4475    YORW  2013-12-01           0.0               -3.7           3.7   
4476    YORW  2014-12-01          21.8               20.8           1.1   
4477    YORW  2015-12-01           6.2                1.7           4.5   
4478    YORW  2016-12-01          10.0                4.9           5.0   
4479    YORW  2017-12-01          11.5                6.2           5.3   
4480    YORW  2018-12-01          13.1                6.3           6.8   
4481    YORW  2019-12-01          18.4               10.4           8.0   
4482     YUM  2011-12-01         518.0              271.0         247.0   
4483     YUM  2012-12-01         179.0             -217.0         396.0   
4484     YUM  2013-12-01        -318.0             -274.0         -44.0   
4485     YUM  2014-12-01        -361.0              294.0        -655.0   
4486     YUM  2015-12-01        -273.0              365.0        -638.0   
4487     YUM  2016-12-01       -2608.0             3982.0       -6590.0   
4488     YUM  2017-12-01        -142.0              577.0        -719.0   
4489     YUM  2018-12-01       -1181.0              411.0       -1592.0   
4490     YUM  2019-12-01        1101.0             1191.0         -90.0   
4491     ZBH  2011-12-01      

In [3]:
# Shuffling the data to create random arrangement of data instances
data = data.sample(frac=1).reset_index(drop=True)
data

Company        Time  Total Assets  Total Liabilities  Total Equity  \
0        SAM  2013-12-01          84.6               27.6          57.0   
1        CMS  2019-12-01        2308.0             2045.0         263.0   
2        HON  2019-12-01         906.0              558.0         348.0   
3       FFIV  2011-09-01         206.3              104.6         101.7   
4        CCL  2018-11-01        1623.0             1396.0         227.0   
5        NEE  2017-12-01        7970.0             3770.0        4200.0   
6        DOV  2014-12-01       -1824.9             -148.2       -1676.7   
7        AGI  2019-12-01         124.3               31.3          93.0   
8         RL  2017-04-01        -561.1             -117.2        -443.9   
9         IP  2013-12-01        -625.0            -2436.0        1811.0   
10       LNC  2016-12-01        9719.0             8858.0         861.0   
11      TSCO  2012-12-01         112.0               95.3          16.7   
12       GTN  2014-12-01         537.2              495.0          42.2   
13       AZO  2013-08-01         626.5              765.7        -139.3   
14      CTXS  2017-12-01        -570.0             1046.2       -1616.2   
15       HPQ  2014-10-01       -2470.0            -1941.0        -529.0   
16        ED  2017-12-01        -144.0            -1263.0        1119.0   
17       NEE  2014-12-01        5299.0             3171.0        2128.0   
18      HAIN  2016-06-01         -91.3              -28.1         -63.2   
19       ROK  2012-09-01         351.6              247.9         103.7   
20       AEE  2015-12-01        1351.0             1118.0         233.0   
21       CMI  2013-12-01        2180.0             1284.0         896.0   
22       AUY  2017-12-01         -38.4               94.3        -132.7   
23      QCOM  2013-09-01        2504.0              -38.0        2542.0   
24       KMX  2020-02-01        2364.3             1952.5         411.9   
25      XLNX  2019-03-01          90.8             -410.4         501.1   
26         F  2011-12-01       13661.0            -2052.0       15713.0   
27      GILD  2015-12-01       17052.0            13771.0        3281.0   
28       DTE  2017-12-01        1726.0             1235.0         491.0   
29       ASH  2013-09-01        -436.0             -960.0         524.0   
...      ...         ...           ...                ...           ...   
4470   JSAIY  2019-03-01        5989.4             6123.5        -134.1   
4471     ADP  2015-06-01        1050.7             2912.4       -1861.7   
4472     SHW  2018-12-01        -765.2             -848.0          82.8   
4473     AME  2015-12-01         239.5              224.4          15.0   
4474     CXO  2013-12-01        1001.8              710.0         291.7   
4475      PH  2018-06-01        -169.8             -768.0         598.2   
4476     XRX  2012-12-01        -101.0              260.0        -361.0   
4477     SJM  2014-04-01          28.4              147.6        -119.2   
4478      GE  2016-12-01     -127888.0          -105294.0      -22594.0   
4479     PFG  2018-12-01      -10905.1            -9729.1       -1176.0   
4480    ORCL  2016-05-01        1277.0             2585.0       -1308.0   
4481    FFIV  2019-09-01         784.8              308.8         476.0   
4482     HRB  2017-04-01        -153.1              -69.1         -84.0   
4483    JBHT  2019-12-01         379.3              213.5         165.6   
4484     OXY  2013-12-01        5233.0             1909.0        3324.0   
4485     ICE  2011-12-01        9505.6             9160.0         345.5   
4486     TAP  2018-12-01        -137.1             -685.6         548.5   
4487    IPGP  2012-12-01         287.4               34.2         253.2   
4488     XRX  2011-12-01        -484.0             -350.0        -134.0   
4489    CPRT  2013-07-01         180.5              -20.8         201.3   
4490     MTD  2013-12-01         130.5               22.7         107.9   
4491     XOM  2012-12-01      

In [4]:
#data.to_csv(r'C:\Users\conor\Documents\MScAIProject\MSc_AI_Project\Data\Testing\DeltaFilings_ChangeInPrice\MergedStatements\HyperparameterTuning\DeltaMF_data.csv', index = False)